# Resource For 2023

In [5]:
import numpy as np
from scipy import linalg

### Matrix Operations:

In [6]:
def matrix_addition(A, B):
    return np.add(A, B)

def matrix_multiplication(A, B):
    return np.matmul(A, B)

def scalar_multiplication(a, A):
    return a * A

def matrix_transposition(A):
    return A.transpose()

### Floating Point Number Systems:

In [7]:
# (B, t, L, U):
def BtLU_number_of_representable_numbers(BtLU):
    B = np.int64(BtLU[0])
    t = np.int64(BtLU[1])
    L = np.int64(BtLU[2])
    U = np.int64(BtLU[3])
    # ± * (B-1)*(B^t-1)*(U-L+1):
    return 2 * (B - 1) * np.power(B, t-1) * (U - L + 1)
#ENDFUNCTION

def BtLU_largest_representable_number(BtLU):
    B = np.int64(BtLU[0])
    t = np.int64(BtLU[1])
    L = np.int64(BtLU[2])
    U = np.int64(BtLU[3])
    mantissa = "0."
    for i in range(0, t):
        mantissa = mantissa + str(B-1)
    
    return np.power(B, U)*np.double(mantissa)
#ENDFUNCTION

def BtLU_smallest_positive_representable_number(BtLU):
    B = np.int64(BtLU[0])
    t = np.int64(BtLU[1])
    L = np.int64(BtLU[2])
    U = np.int64(BtLU[3])
    mantissa = "0."
    for i in range(0, t-1):
        mantissa = mantissa + "0"
    mantissa = mantissa + "1"
    exp = np.double(np.power(B, np.abs(L))) # exp = B^|U|

    if L < 0:
        print(f"1/{B}^{U} * {np.double(mantissa)}")
        return np.reciprocal(exp)*np.double(mantissa)
    else:
        return exp*np.double(mantissa)
    #ENDIF
#ENDFUNCTION

def absolute_error(actual_x, calculated_x):
    return np.abs(calculated_x, actual_x)

def relative_error(actual_x, calculated_x):
    return absolute_error(actual_x, calculated_x) / np.abs(actual_x)

# Get the eps of a numpy datatype:
def get_eps(dtype):
    return np.finfo(dtype).eps

def BtLU_get_eps(BtLU):
    B = np.int64(BtLU[0])
    t = np.int64(BtLU[1])
    L = np.int64(BtLU[2])
    U = np.int64(BtLU[3])

    # eps = 1/2 * B^(1-t) = 1/2 * exp
    exp = np.double(np.power(B, np.abs(1-t)))
    if (1-t) < 0:
        return np.double(0.5) * np.reciprocal(exp)
    else:
        return np.double(0.5) * exp
#ENDFUNCTION

def find_machine_eps(dtype):
    """
    For a given floating point type determine machine epsilon.
    """
    # store values for one and two of correct type
    dtype1 = dtype(1.0)
    dtype2 = dtype(2.0)
    
    # initialise loop
    eps = dtype(1.0)
    eps = eps / dtype2
    eps1 = eps + dtype1
    
    # peform testing loop
    while eps1 > 1:
        eps = eps / dtype2
        eps1 = eps + dtype(1.0)
    
    return eps
#ENDFUNCTION

### Systems of linear equations:

In [8]:
# 
# Solve vector x for which:
#       Ax = b
#  where A is a LOWER TRIANGULAR MATRIX.
#
def solve_where_A_is_LOWER_triangular_matrix(A, b):
    x = np.ones((np.size(b), 1))
    for row in range(0, np.size(A, 1)):
        for column in range(0, row):
            b[row][0] -= A[row][column] * x[column][0]
        #ENDFOR
        x[row][0] = b[row][0] / A[row][row]
    #ENDFOR

    return x
#ENDFUNCTION

def solve_where_A_is_UPPER_triangular_matrix(A, b):
    x = np.ones((np.size(b), 1))
    for row in range(np.size(A, 1)-1, -1, -1):
        for column in range(np.size(A, 1)-1, row, -1):
            b[row][0] -= A[row][column] * x[column][0]
        #ENDFOR
        x[row][0] = b[row][0] / A[row][row]
    #ENDFOR

    return x
#ENDFUNCTION

def gauss_elimination(A, b, verbose=False):
    for column in range(0, np.size(A, 1)-1):
        for row in range(column+1, np.size(A, 1)):
            # [row j] = [row j] - k*[row i]
            k = A[row][column] / A[column][column]
            for i in range(column, np.size(A, 1)):
                A[row][i] = A[row][i] - k*A[column][i]
            #ENDFOR
            b[row][0] = b[row][0] - k*b[column][0]
        #ENDFOR
    #ENDFOR

    return A, b
#ENDFUNCTION

def Gaussian_elimination_pivoting(A, b):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector
    n=len(b)
        
    for i in range(n): # Iterate through the columns
        # find the index of the maximal value in array in column i below A[i,i]
        maximum = abs(A[i,i])
        max_index = i
        for j in range(i+1,n):
            if abs(A[j,i]) > maximum :
                maximum = abs(A[j,i])               
                max_index = j
            #ENDIF
        #ENDFOR
                                       
        '''
        swap row i and row max_index
        '''
        for j in range(0, n):
            temp = A[max_index][j]
            A[max_index][j] = A[i][j]
            A[i][j] = temp
        #ENDFOR

            
        if np.abs(A[i,i])<1.e-15:
            print('A is singular!')
            return    

        '''
        Apply Gaussian elimination to both matrix A and right-hand side vector b
        A becomes an upper triangular matrix at the end of this loop
        '''
        for row in range(i+1, n): # Iterate down the rows below the diagonal.
            # [row j] = [row j] - k*[row i]
            k = A[row][i] / A[i][i]
            for col in range(i, n): # iterate accross the columns.
                A[row][col] = A[row][col] - k*A[i][col]
            #ENDFOR
            b[row] = b[row] - k*b[i]
        #ENDFOR
    #ENDFOR
  
    # solve using previous worksheet
    return solve_where_A_is_UPPER_triangular_matrix(A, b)
#ENDFUNCTION

### LU Factorisation:

In [9]:
def numpy_solver(A, b):
    return np.linalg.solve(A, b)

def LU_factorisation_with_Permutation_matrix(A):
    """
    returns: P, L, U (P => pivot matrix)
    """
    return linalg.lu(A, permute_l=0)
#ENDFUNCTION

### Jacobi Iteration & Gauss-Seidel Iteration:

In [18]:
def Gauss_Seidel_iteration(A, b, max_iteration, x0=None):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    n = len(b)  # dimensions of the linear system of equations

    for i in range(n):
        if np.abs(A[i, i]) < 1.0e-15:
            print("Diagonal element (%f %f)is zero!" % (i, i))
            return
        #ENDIF
    #ENDFOR


    # Create matrix D:
    D = np.zeros((n, n))
    for i in range(0, n):
        D[i][i] = A[i][i]
    #ENDFOR

    # Create matrix L:
    L = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (i < j):
                L[i][j] = A[i][j]
            #ENDIF
        #ENDFOR
    #ENDFOR

    #Create matrix U:
    U = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (i > j):
                U[i][j] = A[i][j]
            #ENDIF
        #ENDFOR
    #ENDFOR

    L_and_U = matrix_addition(L, U) # Calculate L+U

    P = np.zeros([n, n])  # matrix P = D^{-1}(L+U)
    p = np.zeros(n)  # vector p = D^{-1} b

    for i in range(n):
        """
        Compute the matrix P and vector p...
        """
        p[i] = (1/D[i][i]) * b[i]
        P[i] = (1/D[i][i]) * L_and_U[i]
    #ENDFOR
        

    # create a new array to store the results, initialised as zero
    if (x0 != None):
        x = x0
    else:
        x = np.zeros(n)
    #ENDIF

    for it in range(max_iteration):
        """
        Implement Gauss-Seidel iteration -- update x component by component.
        """
        for i in range(0, n):
            Px = np.zeros((n, 1))
            for j in range(0, n):
                for y in range(0, n):
                    Px[j] += P[j][y] * x[y]
                #ENDFOR
                x[j] = p[j] - Px[j]
            #ENDFOR
        #ENDFOR
    #ENDFOR

    return x
#ENDFUNCTION

def Jacobi_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')

    # check diagonal is non zero
    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # construct iteration matrices
    P=np.zeros([n,n])    # matrix P = D^{-1}(L+U)
    p=np.zeros(n)        # vector p = D^{-1} b
    for i in range(n):
        p[i]=b[i]/A[i,i] 
        for j in range(n):
             P[i,j] = A[i,j]/A[i,i]
        P[i,i] = 0
        
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
    
    # perform iteration x <- p - P * x
    for it in range(max_iteration):
        xnew = np.empty_like(x)
        for i in range(n):
            xnew[i] = p[i]
            for j in range(n):
                xnew[i] -= P[i, j] * x[j]
        x = xnew.copy()
                
    return x

def jacobi(A, b, max):
    pass

### Sparse Matrices:

In [11]:
def sparse_matrix_multiplication(A_real, I_row, I_col):
    """
    A_real => array storing nonzero values of matrix A.\n
    I_row  => array storing row position of A_real.\n
    I_col  => array storing column position of A_real.\n
    """
    z = np.zeros((n, 1))
    for k in range(nonzero):
        z[I_row[k]] = z[I_row[k]] + A_real[k] * y[I_col[k]]
    #ENDFOR
    return z
#ENDFUNCTION

### Euclidean Norm:

In [12]:
def magnitude_of_a_vector(r):
    """
    Uses the Euclidean norm to calculate the magnitude of a vector ( ||r|| ).
    """
    x = 0
    for i in range(0, np.size(r, 0)):
        x = x + np.square(r[i])
    #ENDFOR
    return np.sqrt(x)
#ENDFUNCTION

### Freefall:

In [13]:
def freefall(n):
    """
    Plot the trajectory of an object falling freely.
    Input: n number of timesteps
    """

    tfinal = 50.0  # Select the final time
    g = 9.81  # acceleration due to gravity (m/s)
    k = 0.2  # air resistance coefficient

    # initialise time and speed arrays array
    t = np.zeros([n + 1, 1])
    s = np.zeros([n + 1, 1])

    # set initial conditions
    s[0] = 0.0
    t[0] = 0.0

    dt = (tfinal - t[0]) / n  # calculate step size

    # take n time steps, in which it is assumed that the acceleration
    # is constant in each small time interval
    for i in range(n):
        t[i + 1] = t[i] + dt
        s[i + 1] = s[i] + dt * (g - k * s[i])

    # plot output
    plt.plot(t, s, label=f"n = {n}")
#ENDPROCEDURE

### Midpoint Method:

In [14]:
def f(t, y):
    pass
#ENDFUNCTION

def midpoint_method(y, t, dt, n):
    """runs midpoint method on starting values y and t with for n iterations"""
    for i in range(n):
        y_mid = y + 0.5 * dt * f(t, y)
        t_mid = t + 0.5 * dt
        y = y + dt * f(t_mid, y_mid)
        t = t + dt
    #ENDFOR

    return y
#ENDFUNCTION